# The game of Nim

## Part A – do at home before the lab

The purpose of this lab is to implement an optimal algorithm to play the game of Nim. This is an interesting challenge for 2 reasons:

1. Game of Nim is a rare case of a strategy game, which was "solved" in the sense that if an optimal move exists, it can be always quickly computed, and if no optimal move exists, it can be quickly proven to be the case. This is unlike the games of Chess or Go, for which new algorithms constantly appear and it doesn't look like we've invented the optimal algorithm yet.

2. The optimal algorithm to play the game of Nim uses bitwise exclusive or, which is otherwise important in Computer Science & Software Engineering, and you'll have to know it in your further studies & in your future job.

## Description of the game

In the game of Nim two players, Alice and Bob are presented with a finite number of heaps of variable size. For the sake of presentation, let's say 5 heaps:

`Nim:
1: X X X X X
2: X X X X
3: X X X X X
4: X X
5: X X X`

In each turn, a player chooses a row and how many items to remove from the row. It's Alice's turn now, and she decides to remove elements from row 3 and she wants to remove 2 elements:

`Alice: 3, 2
Nim:
1: X X X X X
2: X X X X
3: X X X
4: X X
5: X X X`

It's now time for Bob to play. Bob decides to remove items from row 1, and he wants to remove all 5 items:

`Bob: 1, 5
Nim:
1: 
2: X X X X
3: X X X
4: X X
5: X X X`

The game continues, and eventually it's Alice's turn and the board looks like this:

`Nim:
1: 
2: 
3: 
4: 
5: X X
`

Alice can now remove the last two elements from the last heap. Because she is the last one to make the move, she wins and this is how the game ends.

`Alice: 5, 2
Nim:
1: 
2: 
3: 
4: 
5: 
Alice wins
`

## Programming tasks

Please accomplish the following programming task before your scheduled lab

### Present Board

We will use a Python list to represent the current state of the Nim Board. The list will contain only information about the length of each row. Let us first write a function which will present the board nicely.

```
>>> board = [5, 4, 5, 2, 3, 1]
>>> result = present_board(board)
>>> print(result)
>>> Nim:
1: X X X X X
2: X X X X
3: X X X X X
4: X X
5: X X X
6: X
```

Warning! `board` is not limited to a fixed size of 5 or 6. It can have any size less than 2^32

Pro tip: `"\n".join(["a", "b", "c"])` allows you to join multiple strings efficiently using new line as separator.

In [1]:
def present_board(board):
    rows = ["Nim:"]
    for i, elem_no in enumerate(board):
        elems = []
        elems.append(str(i + 1) + ":")
        for j in range(elem_no):
            elems.append(" X")
        rows.append("".join(elems))
    return "\n".join(rows)

In [2]:
board = [4, 3, 1, 2]
print(present_board(board))
assert(present_board(board) == "Nim:\n1: X X X X\n2: X X X\n3: X\n4: X X")

Nim:
1: X X X X
2: X X X
3: X
4: X X


## Part B – do in the lab

The algorithm to optimally play the game of Nim uses bitwise exclusive-or to work.

Before we understand how that works, we have to implement exclusive-or as a binary operator.

Note: binary operator exclusive-or is not the same as bitwise exclusive-or.

When treated as a binary operator, exclusive-or works only with two inputs, which can take two values: 0 or 1. That's why there are only four possible ways to use a binary operator exclusive-or:

`
exclusive_or(0, 0) → 0
exclusive_or(1, 0) → 1
exclusive_or(0, 1) → 1
exclusive_or(1, 1) → 0
`

Implement `exclusive_or` using `if` statements. **Do not** use python-provided exclusive-or, `^` or equality opertator `==`

In [3]:
def exclusive_or(a, b):
    if a == 0 and b == 0:
        return 0
    if a == 0 and b == 1:
        return 1
    if a == 1 and b == 0:
        return 1
    return 0

In [4]:
assert exclusive_or(0, 0) == 0
assert exclusive_or(0, 1) == 1
assert exclusive_or(1, 0) == 1
assert exclusive_or(1, 1) == 0

In order to implement bitwise exclusive or we have to be able to convert numerals into their binary representations. For this purpose we can use python lists.

Please implement the functions `numeral_binary` and `binary_numeral`

``` 
>>> numeral_binary(6)
>>> [1, 1, 0]
>>> binary_numeral([1, 1, 1]):
>>> 7
```

Pro tip: double forward slash, `//`, denotes integer divison operator in python 3. Integer division ignores remainders, so for example `3//3 → 1` as expected, but `4//3 → 1` and `5//3 → 1` give the same answer, since the remainders of 1 and 2 respectively are both ignored.

In [5]:
def numeral_binary(numeral):
    result = []
    while numeral:
        result.append(numeral%2)
        numeral //= 2
    return result[::-1]

In [6]:
def binary_numeral(binary):
    result = 0
    for i, value in enumerate(binary[::-1]):
        result += value * (2 ** i)
    return result

In [7]:
assert numeral_binary(6) == [1, 1, 0]
assert numeral_binary(1) == [1]
assert numeral_binary(0) == []
assert binary_numeral([1, 1, 0]) == 6
assert binary_numeral([1]) == 1
assert binary_numeral([]) == 0
assert numeral_binary(1789) == [1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1]
assert binary_numeral([1, 1, 0, 1, 0, 1, 1, 1, 1]) == 431
assert binary_numeral(numeral_binary(9183742)) == 9183742

Now you are ready to implement bitwise exclusive or. The idea is that given any two numbers:
1. We'll first find their binary representations (using `binary_numeral`)
2. We'll make each representation to have equal length by padding the shorter representation with 0s on the left.
3. We'll perform exclusive or (using `exclusive_or`) on pairs of corresponding digits.

For example, to compute 3 ^ 5 we will:
1. Represent 3 in binary as [1, 1] and 5 in binary as [1, 0, 1]
2. Pad [1, 1] on the left to get [0, 1, 1]
3. Compute bitwise exclusive or on [0, 1, 1] and [1, 0, 1] to get [1, 1, 0]
4. Represent [1, 1, 0] using arabic numeral

In [8]:
def bitwise_exclusive_or(a, b):
    a = numeral_binary(a)
    b = numeral_binary(b)
    if len(a) < len(b):
        a, b = b, a
    size_diff = len(a) - len(b)
    b = [0] * size_diff + b
    return binary_numeral([exclusive_or(i, j) for i, j in zip(a, b)])

In [9]:
bitwise_exclusive_or(3, 5)

6

As you might have already realised python implements bitwise exclusive or using the caret syntax `^`. Let's use python's native implementation to check our code.

In [10]:
assert bitwise_exclusive_or(3, 5) == 3 ^ 5
assert bitwise_exclusive_or(0, 4) == 4
assert bitwise_exclusive_or(5, 5) == 0
assert bitwise_exclusive_or(17, 18) == 17 ^ 18
assert bitwise_exclusive_or(1234567890, 121212121212) == 1234567890 ^ 121212121212